In [ ]:
import pdfplumber

with pdfplumber.open("Financial Terms.pdf") as pdf:
    first_page = pdf.pages[0]
    print(first_page.chars)

In [ ]:
term_report = pdfplumber.open("Financial Terms.pdf").pages
fin_term_en_cn = {}
for page in term_report:
    tmp_page= page.extract_text(x_tolerance=1, y_tolerance=1).split('\n')
    for term in tmp_page:
        term = term.split()
        if len(term)<2:
            print(term)
        else:
            fin_term_en_cn[term[0]] = ' '.join(term[1:])

In [30]:
import json 
with open('fin_term_cn_2_en.json','w') as f:
    json.dump(fin_term_en_cn,f, indent=3)

In [1]:
import pdfplumber
term_report = pdfplumber.open("Q3_BAIDU_2018.pdf").pages

In [43]:
table_settings = {
    "vertical_strategy": "text", 
    "horizontal_strategy": "lines",
    "explicit_vertical_lines": [],
    "explicit_horizontal_lines": [],
    "snap_tolerance": 1,
    "join_tolerance": 1,
    "edge_min_length": 3,
    "min_words_vertical": 3,
    "min_words_horizontal": 1,
    "keep_blank_chars": False,
    "text_tolerance": 1,
    "text_x_tolerance": None,
    "text_y_tolerance": None,
    "intersection_tolerance": 3,
    "intersection_x_tolerance": None,
    "intersection_y_tolerance": None,
}
table = term_report[13].extract_table(table_settings)

In [2]:
term_report[13].extract_text(x_tolerance=1, y_tolerance=1).split('\n')

[' ',
 'Baidu, Inc. ',
 'Reconciliations of Non-GAAP Financial Measures to the Nearest Comparable GAAP Measures ',
 '      Three months ended    ',
 ' (In RMB millions except for ADS and per ADS  September 30,  June 30,   September 30, ',
 'information, unaudited)     2017       2018         2018    ',
 '             ',
 'Operating income - Baidu Core       5,589   6,736      7,015  ',
 'Add: Share-based compensation expenses       788   1,033      1,035  ',
 'Non-GAAP operating income-Baidu Core       6,377   7,769      8,050  ',
 '             ',
 'Add: Depreciation of fixed assets    868   760      885 ',
 'Add: Amortization of intangible assets    329   33      47 ',
 'Adjusted EBITDA - Baidu Core    7,574   8,562      8,982 ',
 '                        ',
 'Operating loss - iQIYI  (1,067)   (1,328)      (2,591)  ',
 'Add: Share-based compensation expenses       53   84      204  ',
 'Non-GAAP operating loss - iQIYI       (1,014)   (1,244)      (2,387)  ',
 '             ',
 'Add: 